In [1]:
using GeometricIntegrators.Integrators.SPARK
using GeometricIntegrators.Tableaus

┌ Info: Precompiling GeometricIntegrators [dcce2d33-59f6-5b8d-9047-0defad88ae06]
└ @ Base loading.jl:1278


#### Definition of Projective SPARK Methods

$$
\begin{aligned}
P_{n,i} &= p_{n} + h \sum \limits_{j=1}^{s} a^{p}_{ij} F_{n,j} + h \sum \limits_{j=1}^{\tilde{s}} \alpha^{p}_{ij} \tilde{G}_{n,j} && \text{for $i = 1, \, ... , \, s$} , \\
Q_{n,i} &= q_{n} + h \sum \limits_{j=1}^{s} a^{q}_{ij} V_{n,j} + h \sum \limits_{j=1}^{\tilde{s}} \alpha^{q}_{ij} \tilde{\Lambda}_{n,j} && \text{for $i = 1, \, ... , \, s$} , \\
\tilde{P}_{n,i} &= p_{n} + h \sum \limits_{j=1}^{s} \tilde{a}^{p}_{ij} F_{n,j} + h \sum \limits_{j=1}^{\tilde{s}} \tilde{\alpha}^{p}_{ij} \tilde{G}_{n,j} && \text{for $i = 1, \, ... , \, \tilde{s}$} , \\
\tilde{Q}_{n,i} &= q_{n} + h \sum \limits_{j=1}^{\tilde{s}} \tilde{a}^{q}_{ij} V_{n,j} + h \sum \limits_{j=1}^{\tilde{s}} \tilde{\alpha}^{q}_{ij} \tilde{\Lambda}_{n,j} && \text{for $i = 1, \, ... , \, \tilde{s}$} , \\
p_{n+1} &= p_{n} + h \sum \limits_{j=1}^{s} b^{p}_{j} F_{n,j} + h \sum \limits_{j=1}^{\tilde{s}} \beta^{p}_{j} \tilde{G}_{n,j} , \\
q_{n+1} &= q_{n} + h \sum \limits_{j=1}^{s} b^{q}_{j} V_{n,j} + h \sum \limits_{j=1}^{\tilde{s}} \beta^{q}_{j} \tilde{\Lambda}_{n,j} , \\
0 &= \phi(Q_{n,i}, P_{n,i}) && \text{for $i = 1, \, ... , \, s$} , \\
0 &= \sum \limits_{j=1}^{\tilde{s}} \omega_{ij} \, \phi ( \tilde{P}_{n,j}, \tilde{Q}_{n,j}) + \omega_{i\tilde{s}+1} \, \phi (q_{n+1}, p_{n+1}) && \text{for $i = 1, \, ... , \, \tilde{s}-\rho$} , \\
0 &= \sum \limits_{j=1}^{\tilde{s}} \delta_{ij} \, \tilde{\Lambda}_{n,j} && \text{for $i = 1, \, ... , \, \rho$} ,
\end{aligned}
$$

#### Symplecticity Conditions

$$
\begin{aligned}
b^{p}_{i} b^{q}_{j} &= b^{p}_{i} a^{q}_{ij} + b^{q}_{j} a^{p}_{ji} && \text{for $i = 1, \, ... , \, s$, $j = 1, \, ... , \, s$} ,
\\
\beta^{p}_{i} \beta^{q}_{j} &= \beta^{p}_{i} \tilde{\alpha}^{q}_{ij} + \beta^{q}_{j} \tilde{\alpha}^{p}_{ji}  && \text{for $i = 1, \, ... , \, \tilde{s}$, $j = 1, \, ... , \, \tilde{s}$} ,
\\
b^{p}_{i} \beta^{q}_{j} &= b^{p}_{i} \alpha^{q}_{ij} + \beta^{q}_{j} \tilde{a}^{p}_{ji}  && \text{for $i = 1, \, ... , \, s$, $j = 1, \, ... , \, \tilde{s}$} ,
\\
b^{q}_{i} \beta^{p}_{j} &= b^{q}_{i} \alpha^{p}_{ij} + \beta^{p}_{j} \tilde{a}^{q}_{ji} && \text{for $i = 1, \, ... , \, s$, $j = 1, \, ... , \, \tilde{s}$} ,
\\
b^{q}_{i} &= b^{p}_{i}  && \text{for $i = 1, \, ... , \, s$} ,
\\
\beta^{q}_{i} &= \beta^{p}_{i}  && \text{for $i = 1, \, ... , \, \tilde{s}$} ,
\\
0 &= \mathrm{d} \sum \limits_{i=1}^{\tilde{s}} ( \beta^{q}_{i} \, \tilde{P}_{n,i} - \beta^{p}_{i} \, \vartheta (\tilde{Q}_{n,i}) ) \wedge \mathrm{d} \Lambda_{n,i} .
\end{aligned}
$$

#### Construction

- choose symplectic conjugate tableaus for $(a^q, a^p, b^q, b^p)$ and $(\tilde{\alpha}^q, \tilde{\alpha}^p, \beta^q, \beta^p)$
- either
  - choose $(\alpha^q, \alpha^p)$ and compute $(\tilde{a}^q, \tilde{a}^p)$, or
  - choose $(\tilde{a}^q, \tilde{a}^p)$ and compute $(\alpha^q, \alpha^p)$

In [2]:
function compute_conjugate(a, b, b̄)
    ā = zero(a)
    for i in axes(ā,1)
        for j in axes(ā,2)
            ā[i,j] = b̄[j] / b[i] * ( b[i] - a[j,i] )
        end
    end
    return ā
end

compute_conjugate (generic function with 1 method)

In [3]:
function compute_ã(α, β, b)
    s = length(b)
    s̃ = length(β)
    ã = zeros(eltype(α), s̃, s)
    for i in 1:s̃
        for j in 1:s
            ã[i,j] = b[j] / β[i] * ( β[i] - α[j,i] )
        end
    end
    return ã
end

compute_ã (generic function with 1 method)

In [4]:
function get_ã(α_q, β_q, b_q, α_p, β_p, b_p)
    ã_q = compute_ã(α_p, β_p, b_q)
    ã_p = compute_ã(α_q, β_q, b_p)
    return (ã_q, ã_p)
end

get_ã (generic function with 1 method)

In [5]:
function compute_α(ã, b, β)
    s = length(b)
    s̃ = length(β)
    α = zeros(eltype(ã), s, s̃)
    for i in 1:q.s
        for j in 1:s̃
            α[i,j] = β[j] / b[i] * ( b[i] - ã[j,i] )
        end
    end
    return α
end

compute_α (generic function with 1 method)

In [6]:
function get_α(ã_q, b_q, β_q, ã_p, b_p, β_p)
    α_q = compute_α(ã_p, b_p, β_q)
    α_p = compute_α(ã_q, b_q, β_p)
    return (α_q, α_p)
end

get_α (generic function with 1 method)

In [7]:
function check_symplecticity(a_q, a_p, α_q, α_p, a_q̃, a_p̃, α_q̃, α_p̃, b_q, b_p, β_q, β_p; atol=1E-14, rtol=1E-14)
    s_a_qp  = isapprox.(a_p, compute_conjugate(a_q, b_q, b_p); atol=atol, rtol=rtol)
    s_α_q̃p̃  = isapprox.(α_p̃, compute_conjugate(α_q̃, β_q, β_p); atol=atol, rtol=rtol)
    s_αa_q̃p = isapprox.(a_q̃, compute_ã(α_p, β_p, b_q); atol=atol, rtol=rtol)
    s_αa_qp̃ = isapprox.(a_p̃, compute_ã(α_q, β_q, b_p); atol=atol, rtol=rtol)
    s_b_qp  = isapprox.(b_q, b_p; atol=atol, rtol=rtol)
    s_β_qp  = isapprox.(β_q, β_p; atol=atol, rtol=rtol)
    return (s_a_qp, s_α_q̃p̃, s_αa_q̃p, s_αa_qp̃, s_b_qp, s_β_qp)
end

check_symplecticity (generic function with 1 method)

In [8]:
function check_symplecticity(tab::AbstractTableauSPARK; kwargs...)
    check_symplecticity(tab.q.a, tab.p.a,
                        tab.q.α, tab.p.α,
                        tab.q̃.a, tab.p̃.a,
                        tab.q̃.α, tab.p̃.α,
                        tab.q.b, tab.p.b,
                        tab.q.β, tab.p.β;
                        kwargs...)
end

check_symplecticity (generic function with 2 methods)

In [9]:
check_symplecticity(getTableauVSPARKGLRKpInternal(1))

(Bool[1], Bool[0], Bool[0], Bool[0], Bool[1], Bool[1])

In [10]:
check_symplecticity(getTableauVSPARKGLRKpInternal(2))

(Bool[1 1; 1 1], Bool[1 1; 1 1], Bool[1 0; 0 1], Bool[1 0; 0 1], Bool[1, 1], Bool[1, 1])

In [11]:
check_symplecticity(getTableauVSPARKGLRKpInternal(3))

(Bool[1 1 1; 1 1 1; 1 1 1], Bool[0 0 0; 0 0 0; 0 0 0], Bool[0 0 0; 0 0 0; 0 0 0], Bool[0 0 0; 0 0 0; 0 0 0], Bool[1, 1, 1], Bool[1, 1, 1])

In [12]:
check_symplecticity(getTableauVSPARKGLRKpInternal(4))

(Bool[1 1 1 1; 1 1 1 1; 1 1 1 1; 1 1 1 1], Bool[1 1 1 1; 1 1 1 1; 1 1 1 1; 1 1 1 1], Bool[1 0 0 0; 0 1 0 0; 0 0 1 0; 0 0 0 1], Bool[1 0 0 0; 0 1 0 0; 0 0 1 0; 0 0 0 1], Bool[1, 1, 1, 1], Bool[1, 1, 1, 1])

In [36]:
function myTableauVSPARKInternalProjection(name, q, p, d=[]; R∞=1)
    T = eltype(q.a)

    @assert q.s == p.s

    s = q.s
    o = min(q.o, p.o)

    β_q = q.b .* (1 + R∞) / 2
    β_p = p.b .* (1 + R∞) / 2

    α_q = zeros(T, s, s)
    α_p = zeros(T, s, s)

    for i in 1:s
        α_q[i,:] .= q.b ./ 2
        α_p[i,:] .= p.b ./ 2
    end
    
    a_q̃, a_p̃ = get_ã(α_q, β_q, q.b, α_p, β_p, p.b)

    c_λ = q.c
    d_λ = q.b
    ω_λ = zeros(T, 1, s+1)
    δ_λ = zeros(T, s-1, s)
    
    ω_λ[1,s+1] = 1
    
    for i in 1:s-1
        δ_λ[i,i] = +1
        δ_λ[i,s] = -1
    end


    if length(d) == 0
        return TableauVSPARKprimary(name, o,
                            q.a, p.a, α_q, α_p,
                            a_q̃, a_p̃, q.a, p.a,
                            q.b, p.b, β_q, β_p,
                            q.c, p.c, c_λ, d_λ,
                            ω_λ, δ_λ)
    else
        @assert length(d) == q.s == p.s

        return TableauVSPARKprimary(name, o,
                            q.a, p.a, α_q, α_p,
                            a_q̃, a_p̃, q.a, p.a,
                            q.b, p.b, β_q, β_p,
                            q.c, p.c, c_λ, d_λ,
                            ω_λ, δ_λ, d)
    end

end

"Tableau for Gauss-Legendre method with s stages and symplectic projection."
function myTableauVSPARKGLRKpInternal(s)
    glrk = getCoefficientsGLRK(s)
    myTableauVSPARKInternalProjection(Symbol("vpglrk", s, "pInternal"), glrk, glrk; R∞=(-1)^s)
end

myTableauVSPARKGLRKpInternal

In [37]:
check_symplecticity(myTableauVSPARKGLRKpInternal(1))

(Bool[1], Bool[0], Bool[1], Bool[1], Bool[1], Bool[1])

In [38]:
check_symplecticity(myTableauVSPARKGLRKpInternal(2))

(Bool[1 1; 1 1], Bool[1 1; 1 1], Bool[1 1; 1 1], Bool[1 1; 1 1], Bool[1, 1], Bool[1, 1])

In [39]:
check_symplecticity(myTableauVSPARKGLRKpInternal(3))

(Bool[1 1 1; 1 1 1; 1 1 1], Bool[0 0 0; 0 0 0; 0 0 0], Bool[1 1 1; 1 1 1; 1 1 1], Bool[1 1 1; 1 1 1; 1 1 1], Bool[1, 1, 1], Bool[1, 1, 1])

In [40]:
check_symplecticity(myTableauVSPARKGLRKpInternal(4))

(Bool[1 1 1 1; 1 1 1 1; 1 1 1 1; 1 1 1 1], Bool[1 1 1 1; 1 1 1 1; 1 1 1 1; 1 1 1 1], Bool[1 1 1 1; 1 1 1 1; 1 1 1 1; 1 1 1 1], Bool[1 1 1 1; 1 1 1 1; 1 1 1 1; 1 1 1 1], Bool[1, 1, 1, 1], Bool[1, 1, 1, 1])